In [1]:
import joblib
import pandas as pd

### Testing assets

In [10]:
scaler_path = r"../assets/normalizer.save"
encoder_path = r"../assets/encoder.save"
columns_used_path = r"../assets/columns_used.save"
imputer_path = r"../assets/imputer.save"

scaler = joblib.load(scaler_path)
encoder = joblib.load(encoder_path)
columns_used = joblib.load(columns_used_path)
imputer = joblib.load(imputer_path)

### Load testing data

In [2]:
testing_data_path = r"../datasets/DatiumTest.rpt"
testing_df = pd.read_csv(testing_data_path, delimiter="\t")

/var/folders/8b/2xzjkyq14dz6gp0ygflgcj6w0000gn/T/ipykernel_7096/2384415063.py:2: DtypeWarning: Columns (27,91) have mixed types. Specify dtype option on import or set low_memory=False.
  testing_df = pd.read_csv(testing_data_path, delimiter="\t")


In [3]:
testing_df.shape

(11488, 130)

In [6]:
[print(f"{col} : {null_count}") for col, null_count in testing_df.isnull().sum().items() if null_count>0]

Series : 165
SeriesModelYear : 5822
BadgeDescription : 813
BadgeSecondaryDescription : 10157
BodyConfigDescription : 9057
WheelBaseConfig : 11095
Roofline : 11288
ExtraIdentification : 10550
GearLocationDescription : 4
GearNum : 9
CamDescription : 9
FuelCapacity : 15
MethodOfDeliveryDescription : 12
GrossCombinationMAss : 5723
GrossVehicleMass : 3333
VIN : 54
WheelBase : 4
Height : 13
Length : 18
Width : 10
KerbWeight : 1126
TareMass : 1543
PayLoad : 4110
Power : 4
PowerRPMFrom : 11148
PowerRPMTo : 4
Torque : 3
TorqueRPMFrom : 8486
TorqueRPMTo : 12
RonRating : 4138
ModelCode : 1944
ValvesCylinder : 4
EngineConfigurationDescription : 7
EngineNum : 213
Acceleration : 9148
FrontTyreSize : 18
RearTyreSize : 18
FrontRimDesc : 10
RearRimDesc : 10
TowingBrakes : 519
TowingNoBrakes : 627
WarrantyCustAssist : 5952
FreeScheduledService : 11411
WarrantyYears : 14
WarrantyKM : 36
FirstServiceKM : 1759
FirstServiceMonths : 2537
RegServiceMonths : 322
AltEngEngineType : 11190
AltEngBatteryType : 111

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### Select only cols used for features

In [11]:
features_df = testing_df[columns_used].copy()

In [13]:
features_df.shape

(11488, 69)